For this data, we haven't split the train and test yet.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scispacy #https://allenai.github.io/scispacy/
import spacy
import re
from tqdm import tqdm
import string
import os

In [4]:
#this is the path to mimic data if you're reading from a csv. Else uncomment the code to read from database below
MIMIC_NOTES_FILE = './physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv'

df = pd.read_csv(MIMIC_NOTES_FILE)

In [5]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [18]:
# STEP 1: load Note datasets
# update these constants to run this script
preprocessed_data_folder = './Preprocessed_Data/'
if not os.path.exists(preprocessed_data_folder):
    os.mkdir(preprocessed_data_folder)

OUTPUT_DIR =  preprocessed_data_folder#this path will contain tokenized notes. This dir will be the input dir for create_pretrain_data.sh
MIMIC_NOTES_FILE = './physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv' #this is the path to mimic data if you're reading from a csv. Else uncomment the code to read from database below

df_notes = pd.read_csv(MIMIC_NOTES_FILE)

/home/gw1107/.conda/envs/NYU_DL/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# STEP 2: Generate the cohort to pretrain on (here e.g. we only pretrian on XXXX) 
# # select category/ all data
# list0 = ['Echo', 'ECG', 'Rehab Services']
# list1 = ['Nursing', 'Physician ']
# # this is done
# #list2 = ['Case Management ', 'Respiratory ', 'Nutrition', 'General', ]
# list3 = ['Consult']
# list4 = ['Radiology', 'Nursing/other']
# list5 = ['Pharmacy','Social Work']

# number = 0

# df_notes = df_notes[df_notes.CATEGORY.isin(list0)]



# # IMPORTANT: if you fine tune on the same dataset that you use for pretrain, you need to preclude the fine-tune test admissions 
# df_test_ids = pd.read_csv('FINETUNE_DATA_PATH/test.csv').HADM_ID.unique()
# df_notes_fold = df_notes[~df_notes.HADM_ID.isin(df_test_ids)]
df_notes_fold = df_notes[:100].copy()

In [20]:
# STEP 3: Preprocessing
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    # remove bullepoint like number (what if it's 5.5 mg)
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    # these two kind of make sense
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    
    y=re.sub('--|__|==','',y)
    
    # remove all digits, spaces
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocess2(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    # remove bullepoint like number (what if it's 5.5 mg)
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    # these two kind of make sense
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    
    y=re.sub('--|__|==','',y)
    
    # remove all digits, spaces
    y = y.translate(str.maketrans("", ""))
    y = " ".join(y.split())
    return y

def preprocessing(df_notes): 
    df_notes['TEXT']=df_notes['TEXT'].fillna(' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\n',' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\r',' ')
    df_notes['TEXT']=df_notes['TEXT'].apply(str.strip)
    df_notes['TEXT']=df_notes['TEXT'].str.lower()

    df_notes['TEXT']=df_notes['TEXT'].apply(lambda x: preprocess1(x))
    
    return df_notes

df_notes_fold = preprocessing(df_notes_fold)

In [5]:
#setting sentence boundaries
def sbd_component(doc):
    for i, token in enumerate(doc[:-2]):
        # define sentence start if period + titlecase token
        if token.text == '.' and doc[i+1].is_title:
            doc[i+1].sent_start = True
        if token.text == '-' and doc[i+1].text != '-':
            doc[i+1].sent_start = True
    return doc

# STEP 4: Notes to Sentences
#### NOTE THAT HERE we use 'sci' instead of english
# from spacy.lang.en import English
# nlp = English()  # just the language with no model
# nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp = spacy.load('en_core_sci_md', disable=['tagger','ner'])
nlp.add_pipe(sbd_component, before='parser')  

# nlp praser may not work when there is only one token. In these cases, we just remove them as note that has length 1 usually is some random stuff

def toSentence(x):
    doc = nlp(x)
    text=[]
    try:
        for sent in doc.sents:
            st=str(sent).strip() 
            if len(st)<20:
                #a lot of abbreviation is segmented as one line. But these are all describing the previous things
                #so I attached it to the sentence before
                if len(text)!=0:
                    text[-1]=' '.join((text[-1],st))
                else:
                    text=[st]
            else:
                text.append((st))
    except:
        print(doc)
    return text

pretrain_sent=df_notes_fold['TEXT'].apply(lambda x: toSentence(x))

In [ ]:
# STEP 5: Create Pretraining File
number = "all_withnumber"
numer = 'test'
file=open(OUTPUT_DIR + 'clinical_sentences_category_{}.txt'.format(number),'w')
pretrain_sent_value = pretrain_sent.values
for i in tqdm(range(len(pretrain_sent))):
    if len(pretrain_sent_value[i]) > 0:
        # remove the one token note
        note = pretrain_sent_value[i]
        for sent in note:
            file.write(sent+'\n')
        file.write('\n')

In [21]:
df_notes_fold['TEXT'][98]

'admission date: discharge date: date of birth: sex: m service: medicine allergies: doxepin / levofloxacin / oxycontin / benzodiazepines / ativan attending: chief complaint: hypotension major surgical or invasive procedure: right internal jugular central venous catheter placement tracheal tube change history of present illness: year-old gentleman with mmp, history of lung cancer s/p right pneumonectomy in , severe copd, recently discharged from micu s/p tracheostomy and placement after admission for respiratory failure due to pneumonia, now re-admitted to with hypotension, tachycardia and uti. . in the ed, rij placed, he was given l ivf and started on ctx. ct head no ich, cxr ? infiltrate vs chf. cta neg for pe (prelim) with clear lung fields, no infiltrates. past medical history: . squamous cell lung carcinoma, status post right pneumonectomy in . . prostate cancer, status post radical prostatectomy. . perioperative pulmonary embolus . . type diabetes mellitus. . chronic obstructive p

In [23]:
a = df_notes['TEXT'][98]

In [29]:
print(a)

Admission Date:  [**2179-3-21**]              Discharge Date:   [**2179-3-26**]

Date of Birth:  [**2114-2-8**]             Sex:   M

Service: MEDICINE

Allergies:
Doxepin / Levofloxacin / Oxycontin / Benzodiazepines / Ativan

Attending:[**Last Name (NamePattern1) 293**]
Chief Complaint:
Hypotension

Major Surgical or Invasive Procedure:
Right Internal Jugular Central Venous Catheter Placement [**2179-3-21**]
Tracheal Tube Change


History of Present Illness:
64 year-old gentleman with MMP, history of lung cancer s/p right
pneumonectomy in [**2174**], severe COPD, recently discharged from
[**Hospital1 18**] MICU [**2178-12-2**] s/p tracheostomy and [**Month/Day/Year 282**] placement after
admission for respiratory failure due to pneumonia, now
re-admitted to [**Hospital1 18**] with  hypotension, tachycardia and UTI.
.

In the ED, RIJ placed, he was given 4L IVF and started on CTX.
CT head no ICH, CXR ? infiltrate vs CHF. CTA neg for PE (prelim)
with clear lung fields, no infiltrates.



In [30]:
print(re.sub('--|__|==','',a))


Admission Date:  [**2179-3-21**]              Discharge Date:   [**2179-3-26**]

Date of Birth:  [**2114-2-8**]             Sex:   M

Service: MEDICINE

Allergies:
Doxepin / Levofloxacin / Oxycontin / Benzodiazepines / Ativan

Attending:[**Last Name (NamePattern1) 293**]
Chief Complaint:
Hypotension

Major Surgical or Invasive Procedure:
Right Internal Jugular Central Venous Catheter Placement [**2179-3-21**]
Tracheal Tube Change


History of Present Illness:
64 year-old gentleman with MMP, history of lung cancer s/p right
pneumonectomy in [**2174**], severe COPD, recently discharged from
[**Hospital1 18**] MICU [**2178-12-2**] s/p tracheostomy and [**Month/Day/Year 282**] placement after
admission for respiratory failure due to pneumonia, now
re-admitted to [**Hospital1 18**] with  hypotension, tachycardia and UTI.
.

In the ED, RIJ placed, he was given 4L IVF and started on CTX.
CT head no ICH, CXR ? infiltrate vs CHF. CTA neg for PE (prelim)
with clear lung fields, no infiltrates.



In [28]:
y

'Admission Date:                Discharge Date:   \n\nDate of Birth:               Sex:   M\n\nService: MEDICINE\n\nAllergies:\nDoxepin / Levofloxacin / Oxycontin / Benzodiazepines / Ativan\n\nAttending:\nChief Complaint:\nHypotension\n\nMajor Surgical or Invasive Procedure:\nRight Internal Jugular Central Venous Catheter Placement \nTracheal Tube Change\n\n\nHistory of Present Illness:\n64 year-old gentleman with MMP, history of lung cancer s/p right\npneumonectomy in , severe COPD, recently discharged from\n MICU  s/p tracheostomy and  placement after\nadmission for respiratory failure due to pneumonia, now\nre-admitted to  with  hypotension, tachycardia and UTI.\n.\n\nIn the ED, RIJ placed, he was given 4L IVF and started on CTX.\nCT head no ICH, CXR ? infiltrate vs CHF. CTA neg for PE (prelim)\nwith clear lung fields, no infiltrates.\n\nPast Medical History:\n Squamous cell lung carcinoma, status post right\npneumonectomy in .\n Prostate cancer, status post radical prostatectomy.\n

In [ ]:
# STEP 6: Download the Implementations and Initial Checkpoint with the SentencePiece Model and Vocab 
# Github Repo:
# XLNet: git clone https://github.com/zihangdai/xlnet.git
# BERT: git clone https://github.com/google-research/bert.git
# Model: 
# XLNet-Base: https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
# BERT-Base-Uncased: https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [ ]:
# STEP 7: Generate Pretraining Tensorflow TF_Records

# For Clinical BERT
# cd to the git repo

# Generate datasets for 128 max seq
!python create_pretraining_data.py \
  --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain.txt \
  --output_file=PRETRAIN_DATA_PATH/tf_examples_128.tfrecord \
  --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=3

# Generate datasets for 512 max seq
!python create_pretraining_data.py \
  --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain.txt \
  --output_file=PRETRAIN_DATA_PATH/tf_examples_512.tfrecord \
  --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=76 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=3

# For Clinical XLNet

!python data_utils.py \
    --bsz_per_host=6 \
    --num_core_per_host=1 \
    --seq_len=512 \
    --reuse_len=256 \
    --input_glob=/scratch/kh2383/MechVent/data/clinical_sentences_pretrain_xlnet.txt \
    --save_dir=/scratch/kh2383/MechVent/data/xlnet_tfrecord/ \
    --num_passes=5 \
    --bi_data=True \
    --sp_path=/scratch/kh2383/clibert/xlnet_cased_L-12_H-768_A-12/spiece.model \
    --mask_alpha=6 \
    --mask_beta=1 \
    --num_predict=85

In [ ]:
# STEP 8: Pretraining Use Original TF implementation

# For Clinical BERT

# First pretrain 100000 steps on the max seq length of 128
!python run_pretraining.py \
  --input_file=PRETRAIN_DATA_PATH/tf_examples_128.tfrecord \
  --output_dir=PRETRAINED_MODEL_PATH/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=INITIAL_DATA_PATH/bert_config.json \
  --init_checkpoint=INITIAL_DATA_PATH/bert_model.ckpt \
  --train_batch_size=64 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=100000 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5

# Then further pretrain 100000 steps on the max seq length of 512
# NOTE: the init_checkpoint should switch to the 128 pretrained model

!python run_pretraining.py \
  --input_file=PRETRAIN_DATA_PATH/tf_examples_512.tfrecord \
  --output_dir=PRETRAINED_MODEL_PATH/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=INITIAL_DATA_PATH/bert_config.json \
  --init_checkpoint=PRETRAINED_MODEL_PATH/pretraining_output_128/model.ckpt-100000 \
  --train_batch_size=16 \
  --max_seq_length=512 \
  --max_predictions_per_seq=76 \
  --num_train_steps=100000 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5


# For Clinical XLNet

# Pretrain for 200000 steps

!python train_gpu.py \
    --record_info_dir=PRETRAIN_DATA_PATH/xlnet_tfrecord/tfrecords/ \
    --model_dir=PRETRAINED_MODEL_PATH/xlnet_model/ \
    --init_checkpoint=INITIAL_DATA_PATH/xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt \
    --train_batch_size=8 \
    --seq_len=512 \
    --reuse_len=256 \
    --mem_len=384 \
    --perm_size=256 \
    --n_layer=12 \
    --d_model=768 \
    --d_embed=768 \
    --n_head=12 \
    --d_head=64 \
    --d_inner=3072 \
    --untie_r=True \
    --mask_alpha=6 \
    --mask_beta=1 \
    --num_predict=85 \
    --num_hosts=1 \
    --num_core_per_host=2 \
    --train_steps=200000 \
    --iterations=500 \
    --save_steps=5000 